In [ ]:
%load_ext blackcellmagic

## Do pre-processing

In [ ]:
import numpy as np
import pandas as pd
import string
from string import punctuation

In [95]:
data_txt = open("data/shakespeare.txt")

In [ ]:
def load_data():

    data_txt = open("data/shakespeare.txt")
    quatrain = []
    volta = []
    couplet = []

    Lines = data_txt.readlines()
    count = 0
    for i, line in enumerate(Lines):
        if any(char.isdigit() for char in line):
            for j in range(1, 9):
                quatrain.append(Lines[i + j][:-1])
            for j in range(9, 13):
                volta.append(Lines[i + j][:-1])
            for j in range(13, 15):
                couplet.append(Lines[i + j][:-1])
    return quatrain, volta, couplet


def load_key(includePunctuation=True):

    data_key = open("data/Syllable_dictionary.txt")
    Lines = data_key.readlines()

    key_list = []

    for i, line in enumerate(Lines):
        split_str = line.strip(punctuation).split()
        if len(split_str) == 3:
            if len(split_str[1]) == 2:
                key_list.append(
                    [split_str[0], [int(split_str[2])], int(split_str[1][1])]
                )
            elif len(split_str[2]) == 2:
                key_list.append(
                    [split_str[0], [int(split_str[1])], int(split_str[2][1])]
                )
            else:
                key_list.append(
                    [split_str[0], [int(split_str[1]), int(split_str[2])], 0]
                )

        else:
            key_list.append([split_str[0], [int(split_str[1])], 0])

    # append punctuation
    if includePunctuation == True:
        # comma
        key_list.append([",", [0], 0])
        # period
        key_list.append([".", [0], 0])
        # question
        key_list.append(["?", [0], 0])
        # colon
        key_list.append([":", [0], 0])
        # semicolon
        key_list.append([";", [0], 0])

    return key_list


def convert_to_nums(dataset, key_list, includePunctuation=True):
    punc_list = [",", ".", "?", ":", ";"]
    word_list = [i[0] for i in key_list]
    seq_list = []
    for i in range(0, len(dataset)):
        words = dataset[i].split()
        seq = []
        for j in range(0, len(words)):
            if words[j].lower() == "th'" or words[j].lower() == "t'":
                n = word_list.index(words[j].lower())
            else:
                n = word_list.index(words[j].lower().strip(punctuation))
            seq.append(n)

            if includePunctuation == True:
                if words[j][-1] in punc_list:
                    seq.append(word_list.index(words[j][-1]))

        seq_list.append(seq)

    return seq_list


def convert_to_nums_red_keylist(dataset, key_list, includePunctuation=True):
    punc_list = [",", ".", "?", ":", ";"]
    word_list = [i[0] for i in key_list]
    seq_list = []

    key_list_red = []
    used_inds = []
    for i in range(0, len(dataset)):
        words = dataset[i].split()
        seq = []
        for j in range(0, len(words)):
            if words[j].lower() == "th'" or words[j].lower() == "t'":
                n = word_list.index(words[j].lower())
            else:
                n = word_list.index(words[j].lower().strip(punctuation))
            if n not in used_inds:
                used_inds.append(n)
                key_list_red.append(key_list[n])

            seq.append(n)
            if includePunctuation == True:
                if words[j][-1] in punc_list:
                    punc_indx = word_list.index(words[j][-1])
                    seq.append(punc_indx)
                    if punc_indx not in used_inds:
                        used_inds.append(punc_indx)
                        key_list_red.append(key_list[punc_indx])
        seq_list.append(seq)

    # convert sequence to 'reduced' indices
    seq_list_red = []
    for i in range(0, len(seq_list)):
        seq_list_red_cur = []
        for j in range(0, len(seq_list[i])):
            seq_list_red_cur.append(used_inds.index(seq_list[i][j]))
        seq_list_red.append(seq_list_red_cur)

    return seq_list_red, key_list_red


def generate_sequences():
    # CALL THIS TO GENERATE THE 3 TRAINING DATASETS
    # OUTPUTS:
    # seq_list_quatrain: list of sequences corresponding to lines in the first 2 quatrains
    # seq_list_volta: list of sequences corresponding to lines in the volta
    # seq_list_couplet: list of sequences corresponding to lines in the couplet
    # key_list: list of lists containing information stored in Syllable_dictionary:
    # Each list corresponds to a word, with index 0 being the word,
    # index 1 being a list containing possible numbers of syllables, and index 2 being the number of syllables if it's
    # an end word.

    quatrain, volta, couplet = load_data()
    key_list = load_key()
    seq_list_quatrain = convert_to_nums(quatrain, key_list)
    seq_list_volta = convert_to_nums(volta, key_list)
    seq_list_couplet = convert_to_nums(couplet, key_list)
    return seq_list_quatrain, seq_list_volta, seq_list_couplet, key_list


def generate_sequences_reduced(includePunctuation=True):
    # CALL THIS TO GENERATE THE 3 TRAINING DATASETS
    # OUTPUTS:
    # seq_lists: list with 3 elements corresponding to the 3 lists of sequences
    # key_lists: list with 3 elements corresponding to the 3 associated key lists

    quatrain, volta, couplet = load_data()
    key_list = load_key(includePunctuation)

    seq_list_quatrain, key_list_quatrain = convert_to_nums_red_keylist(
        quatrain, key_list, includePunctuation
    )
    seq_list_volta, key_list_volta = convert_to_nums_red_keylist(
        volta, key_list, includePunctuation
    )
    seq_list_couplet, key_list_couplet = convert_to_nums_red_keylist(
        couplet, key_list, includePunctuation
    )

    seq_lists = [seq_list_quatrain, seq_list_volta, seq_list_couplet]
    key_lists = [key_list_quatrain, key_list_volta, key_list_couplet]
    return seq_lists, key_lists


def generate_rhyme_lists():
    # Returns 3 lists of rhymes, one for quatrain, volta, couplet

    key_list = load_key(includePunctuation=False)
    seq_lists, key_lists = generate_sequences_reduced(includePunctuation=False)
    rhymes = get_rhymes(seq_lists)

    rhyme_list_quatrain = []
    rhyme_list_volta = []
    rhyme_list_couplet = []
    for i in range(0, len(rhymes[0])):
        rhyme_list_quatrain.append(
            [key_lists[0][rhymes[0][i][0]][0], key_lists[0][rhymes[0][i][1]][0]]
        )
    for i in range(0, len(rhymes[1])):
        rhyme_list_volta.append(
            [key_lists[1][rhymes[1][i][0]][0], key_lists[1][rhymes[1][i][1]][0]]
        )
    for i in range(0, len(rhymes[2])):
        rhyme_list_couplet.append(
            [key_lists[2][rhymes[2][i][0]][0], key_lists[2][rhymes[2][i][1]][0]]
        )

    return [rhyme_list_quatrain, rhyme_list_volta, rhyme_list_couplet]


def get_rhymes(seq_lists):
    # requires sequence lists with no punctuation
    rhyme_list_quatrain = []
    for i in range(0, len(seq_lists[0]) - 2, 4):
        add_rhyme([seq_lists[0][i][-1], seq_lists[0][i + 2][-1]], rhyme_list_quatrain)
        add_rhyme(
            [seq_lists[0][i + 1][-1], seq_lists[0][i + 3][-1]], rhyme_list_quatrain
        )
    rhyme_list_volta = []
    for i in range(0, len(seq_lists[1]) - 2, 4):
        add_rhyme([seq_lists[1][i][-1], seq_lists[1][i + 2][-1]], rhyme_list_volta)
        add_rhyme([seq_lists[1][i + 1][-1], seq_lists[1][i + 3][-1]], rhyme_list_volta)
    rhyme_list_couplet = []

    for i in range(0, len(seq_lists[2]) - 1, 2):
        if seq_lists[2][i] and seq_lists[2][i + 1]:
            add_rhyme(
                [seq_lists[2][i][-1], seq_lists[2][i + 1][-1]], rhyme_list_couplet
            )

    return [rhyme_list_quatrain, rhyme_list_volta, rhyme_list_couplet]


def add_rhyme(rhyme, rhyme_list):
    if rhyme not in rhyme_list and [rhyme[1], rhyme[0]] not in rhyme_list:
        rhyme_list.append(rhyme)

## Unsupervised HMM

In [ ]:
import os
import re
import random
import collections
from tqdm import tqdm
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from matplotlib import animation
from matplotlib.animation import FuncAnimation

In [ ]:
class HiddenMarkovModel:
    """
    Class implementation of Hidden Markov Models.
    """

    def __init__(self, A, O):
        """
        Initializes an HMM. Assumes the following:
            - States and observations are integers starting from 0. 
            - There is a start state (see notes on A_start below). There
              is no integer associated with the start state, only
              probabilities in the vector A_start.
            - There is no end state. 
        Arguments:
            A:          Transition matrix with dimensions L x L.
                        The (i, j)^th element is the probability of
                        transitioning from state i to state j. Note that
                        this does not include the starting probabilities.
            O:          Observation matrix with dimensions L x D.
                        The (i, j)^th element is the probability of
                        emitting observation j given state i.
        Parameters:
            L:          Number of states.
            D:          Number of observations.
            
            A:          The transition matrix.
            
            O:          The observation matrix.
            
            A_start:    Starting transition probabilities. The i^th element
                        is the probability of transitioning from the start
                        state to state i. For simplicity, we assume that
                        this distribution is uniform.
        """

        self.L = len(A)
        self.D = len(O[0])
        self.A = A
        self.O = O
        self.A_start = [1.0 / self.L for _ in range(self.L)]

    def viterbi(self, x):
        """
        Uses the Viterbi algorithm to find the max probability state 
        sequence corresponding to a given input sequence.
        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.
        Returns:
            max_seq:    Output sequence corresponding to x with the highest
                        probability.
        """

        M = len(x)  # Length of sequence.

        # The (i, j)^th elements of probs and seqs are the max probability
        # of the prefix of length i ending in state j and the prefix
        # that gives this probability, respectively.
        #
        # For instance, probs[1][0] is the probability of the prefix of
        # length 1 ending in state 0.
        probs = [[0.0 for _ in range(self.L)] for _ in range(M + 1)]
        seqs = [["" for _ in range(self.L)] for _ in range(M + 1)]

        # Calculate initial prefixes and probabilities.
        for curr in range(self.L):
            probs[1][curr] = self.A_start[curr] * self.O[curr][x[0]]
            seqs[1][curr] = str(curr)

        # Calculate best prefixes and probabilities throughout sequence.
        for t in range(2, M + 1):
            # Iterate over all possible current states.
            for curr in range(self.L):
                max_prob = float("-inf")
                max_prefix = ""

                # Iterate over all possible previous states to find one
                # that would maximize the probability of the current state.
                for prev in range(self.L):
                    curr_prob = (
                        probs[t - 1][prev] * self.A[prev][curr] * self.O[curr][x[t - 1]]
                    )

                    # Continually update max probability and prefix.
                    if curr_prob >= max_prob:
                        max_prob = curr_prob
                        max_prefix = seqs[t - 1][prev]

                # Store the max probability and prefix.
                probs[t][curr] = max_prob
                seqs[t][curr] = max_prefix + str(curr)

        # Find the index of the max probability of a sequence ending in x^M
        # and the corresponding output sequence.
        max_i = max(enumerate(probs[-1]), key=lambda x: x[1])[0]
        max_seq = seqs[-1][max_i]

        return max_seq

    def forward(self, x, normalize=False):
        """
        Uses the forward algorithm to calculate the alpha probability
        vectors corresponding to a given input sequence.
        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.
            normalize:  Whether to normalize each set of alpha_j(i) vectors
                        at each i. This is useful to avoid underflow in
                        unsupervised learning.
        Returns:
            alphas:     Vector of alphas.
                        The (i, j)^th element of alphas is alpha_j(i),
                        i.e. the probability of observing prefix x^1:i
                        and state y^i = j.
                        e.g. alphas[1][0] corresponds to the probability
                        of observing x^1:1, i.e. the first observation,
                        given that y^1 = 0, i.e. the first state is 0.
        """

        M = len(x)  # Length of sequence.
        alphas = [[0.0 for _ in range(self.L)] for _ in range(M + 1)]

        # Note that alpha_j(0) is already correct for all j's.
        # Calculate alpha_j(1) for all j's.
        for curr in range(self.L):
            alphas[1][curr] = self.A_start[curr] * self.O[curr][x[0]]

        # Calculate alphas throughout sequence.
        for t in range(1, M):
            # Iterate over all possible current states.
            for curr in range(self.L):
                prob = 0

                # Iterate over all possible previous states to accumulate
                # the probabilities of all paths from the start state to
                # the current state.
                for prev in range(self.L):
                    prob += alphas[t][prev] * self.A[prev][curr] * self.O[curr][x[t]]

                # Store the accumulated probability.
                alphas[t + 1][curr] = prob

            if normalize:
                norm = sum(alphas[t + 1])
                for curr in range(self.L):
                    alphas[t + 1][curr] /= norm

        return alphas

    def backward(self, x, normalize=False):
        """
        Uses the backward algorithm to calculate the beta probability
        vectors corresponding to a given input sequence.
        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.
            normalize:  Whether to normalize each set of alpha_j(i) vectors
                        at each i. This is useful to avoid underflow in
                        unsupervised learning.
        Returns:
            betas:      Vector of betas.
                        The (i, j)^th element of betas is beta_j(i), i.e.
                        the probability of observing prefix x^(i+1):M and
                        state y^i = j.
                        e.g. betas[M][0] corresponds to the probability
                        of observing x^M+1:M, i.e. no observations,
                        given that y^M = 0, i.e. the last state is 0.
        """

        M = len(x)  # Length of sequence.
        betas = [[0.0 for _ in range(self.L)] for _ in range(M + 1)]

        # Initialize initial betas.
        for curr in range(self.L):
            betas[-1][curr] = 1

        # Calculate betas throughout sequence.
        for t in range(-1, -M - 1, -1):
            # Iterate over all possible current states.
            for curr in range(self.L):
                prob = 0

                # Iterate over all possible next states to accumulate
                # the probabilities of all paths from the end state to
                # the current state.
                for nxt in range(self.L):
                    if t == -M:
                        prob += betas[t][nxt] * self.A_start[nxt] * self.O[nxt][x[t]]

                    else:
                        prob += betas[t][nxt] * self.A[curr][nxt] * self.O[nxt][x[t]]

                # Store the accumulated probability.
                betas[t - 1][curr] = prob

            if normalize:
                norm = sum(betas[t - 1])
                for curr in range(self.L):
                    betas[t - 1][curr] /= norm

        return betas

    def supervised_learning(self, X, Y):
        """
        Trains the HMM using the Maximum Likelihood closed form solutions
        for the transition and observation matrices on a labeled
        datset (X, Y). Note that this method does not return anything, but
        instead updates the attributes of the HMM object.
        Arguments:
            X:          A dataset consisting of input sequences in the form
                        of lists of variable length, consisting of integers 
                        ranging from 0 to D - 1. In other words, a list of
                        lists.
            Y:          A dataset consisting of state sequences in the form
                        of lists of variable length, consisting of integers 
                        ranging from 0 to L - 1. In other words, a list of
                        lists.
                        Note that the elements in X line up with those in Y.
        """

        # Calculate each element of A using the M-step formulas.
        for curr in range(self.L):
            for nxt in range(self.L):
                num = 0.0
                den = 0.0

                for i in range(len(X)):
                    x = X[i]
                    y = Y[i]
                    M = len(x)

                    num += len(
                        [1 for i in range(M - 1) if y[i] == curr and y[i + 1] == nxt]
                    )
                    den += len([1 for i in range(M - 1) if y[i] == curr])

                self.A[curr][nxt] = num / den

        # Calculate each element of O using the M-step formulas.
        for curr in range(self.L):
            for xt in range(self.D):
                num = 0.0
                den = 0.0

                for i in range(len(X)):
                    x = X[i]
                    y = Y[i]
                    M = len(x)

                    num += len([1 for i in range(M) if y[i] == curr and x[i] == xt])
                    den += len([1 for i in range(M) if y[i] == curr])

                self.O[curr][xt] = num / den

    def unsupervised_learning(self, X, N_iters):
        """
        Trains the HMM using the Baum-Welch algorithm on an unlabeled
        datset X. Note that this method does not return anything, but
        instead updates the attributes of the HMM object.
        Arguments:
            X:          A dataset consisting of input sequences in the form
                        of lists of length M, consisting of integers ranging
                        from 0 to D - 1. In other words, a list of lists.
            N_iters:    The number of iterations to train on.
        """

        # Note that a comment starting with 'E' refers to the fact that
        # the code under the comment is part of the E-step.

        # Similarly, a comment starting with 'M' refers to the fact that
        # the code under the comment is part of the M-step.

        for iteration in tqdm(range(1, N_iters + 1)):
            # if iteration % 10 == 0:
            #    print("Iteration: " + str(iteration))

            # Numerator and denominator for the update terms of A and O.
            A_num = [[0.0 for i in range(self.L)] for j in range(self.L)]
            O_num = [[0.0 for i in range(self.D)] for j in range(self.L)]
            A_den = [0.0 for i in range(self.L)]
            O_den = [0.0 for i in range(self.L)]

            # For each input sequence:
            for x in X:
                M = len(x)
                # Compute the alpha and beta probability vectors.
                alphas = self.forward(x, normalize=True)
                betas = self.backward(x, normalize=True)

                # E: Update the expected observation probabilities for a
                # given (x, y).
                # The i^th index is P(y^t = i, x).
                for t in range(1, M + 1):
                    P_curr = [0.0 for _ in range(self.L)]

                    for curr in range(self.L):
                        P_curr[curr] = alphas[t][curr] * betas[t][curr]

                    # Normalize the probabilities.
                    norm = sum(P_curr)
                    for curr in range(len(P_curr)):
                        P_curr[curr] /= norm

                    for curr in range(self.L):
                        if t != M:
                            A_den[curr] += P_curr[curr]
                        O_den[curr] += P_curr[curr]
                        O_num[curr][x[t - 1]] += P_curr[curr]

                # E: Update the expectedP(y^j = a, y^j+1 = b, x) for given (x, y)
                for t in range(1, M):
                    P_curr_nxt = [[0.0 for _ in range(self.L)] for _ in range(self.L)]

                    for curr in range(self.L):
                        for nxt in range(self.L):
                            P_curr_nxt[curr][nxt] = (
                                alphas[t][curr]
                                * self.A[curr][nxt]
                                * self.O[nxt][x[t]]
                                * betas[t + 1][nxt]
                            )

                    # Normalize:
                    norm = 0
                    for lst in P_curr_nxt:
                        norm += sum(lst)
                    for curr in range(self.L):
                        for nxt in range(self.L):
                            P_curr_nxt[curr][nxt] /= norm

                    # Update A_num
                    for curr in range(self.L):
                        for nxt in range(self.L):
                            A_num[curr][nxt] += P_curr_nxt[curr][nxt]

            for curr in range(self.L):
                for nxt in range(self.L):
                    self.A[curr][nxt] = A_num[curr][nxt] / A_den[curr]

            for curr in range(self.L):
                for xt in range(self.D):
                    self.O[curr][xt] = O_num[curr][xt] / O_den[curr]

    def generate_emission(self, M):
        """
        Generates an emission of length M, assuming that the starting state
        is chosen uniformly at random. 
        Arguments:
            M:          Length of the emission to generate.
        Returns:
            emission:   The randomly generated emission as a list.
            states:     The randomly generated states as a list.
        """

        emission = []
        state = random.choice(range(self.L))
        states = []

        for t in range(M):
            # Append state.
            states.append(state)

            # Sample next observation.
            rand_var = random.uniform(0, 1)
            next_obs = 0

            while rand_var > 0:
                rand_var -= self.O[state][next_obs]
                next_obs += 1

            next_obs -= 1
            emission.append(next_obs)

            # Sample next state.
            rand_var = random.uniform(0, 1)
            next_state = 0

            while rand_var > 0:
                rand_var -= self.A[state][next_state]
                next_state += 1

            next_state -= 1
            state = next_state

        return emission, states

    def probability_alphas(self, x):
        """
        Finds the maximum probability of a given input sequence using
        the forward algorithm.
        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.
        Returns:
            prob:       Total probability that x can occur.
        """

        # Calculate alpha vectors.
        alphas = self.forward(x)

        # alpha_j(M) gives the probability that the output sequence ends
        # in j. Summing this value over all possible states j gives the
        # total probability of x paired with any output sequence, i.e. the
        # probability of x.
        prob = sum(alphas[-1])
        return prob

    def probability_betas(self, x):
        """
        Finds the maximum probability of a given input sequence using
        the backward algorithm.
        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.
        Returns:
            prob:       Total probability that x can occur.
        """

        betas = self.backward(x)

        # beta_j(0) gives the probability of the output sequence. Summing
        # this over all states and then normalizing gives the total
        # probability of x paired with any output sequence, i.e. the
        # probability of x.
        prob = sum(
            [betas[1][k] * self.A_start[k] * self.O[k][x[0]] for k in range(self.L)]
        )

        return prob


def supervised_HMM(X, Y):
    """
    Helper function to train a supervised HMM. The function determines the
    number of unique states and observations in the given data, initializes
    the transition and observation matrices, creates the HMM, and then runs
    the training function for supervised learning.
    Arguments:
        X:          A dataset consisting of input sequences in the form
                    of lists of variable length, consisting of integers 
                    ranging from 0 to D - 1. In other words, a list of lists.
        Y:          A dataset consisting of state sequences in the form
                    of lists of variable length, consisting of integers 
                    ranging from 0 to L - 1. In other words, a list of lists.
                    Note that the elements in X line up with those in Y.
    """

    # Make a set of observations.
    observations = set()
    for x in X:
        observations |= set(x)

    # Make a set of states.
    states = set()
    for y in Y:
        states |= set(y)

    # Compute L and D.
    L = len(states)
    D = len(observations)

    # Randomly initialize and normalize matrices A and O.
    A = [[random.random() for i in range(L)] for j in range(L)]

    for i in range(len(A)):
        norm = sum(A[i])
        for j in range(len(A[i])):
            A[i][j] /= norm

    O = [[random.random() for i in range(D)] for j in range(L)]

    for i in range(len(O)):
        norm = sum(O[i])
        for j in range(len(O[i])):
            O[i][j] /= norm

    # Train an HMM with labeled data.
    HMM = HiddenMarkovModel(A, O)
    HMM.supervised_learning(X, Y)

    return HMM


def unsupervised_HMM(X, n_states, N_iters, rng=np.random.RandomState(1)):
    """
    Helper function to train an unsupervised HMM. The function determines the
    number of unique observations in the given data, initializes
    the transition and observation matrices, creates the HMM, and then runs
    the training function for unsupervised learing.
    Arguments:
        X:          A dataset consisting of input sequences in the form
                    of lists of variable length, consisting of integers 
                    ranging from 0 to D - 1. In other words, a list of lists.
        n_states:   Number of hidden states to use in training.
        
        N_iters:    The number of iterations to train on.
        rng:        The random number generator for reproducible result.
                    Default to RandomState(1).
    """

    # Make a set of observations.
    observations = set()
    for x in X:
        observations |= set(x)

    # Compute L and D.
    L = n_states
    D = len(observations)
    # print(D)

    # Randomly initialize and normalize matrices A.
    A = [[rng.random() for i in range(L)] for j in range(L)]

    for i in range(len(A)):
        norm = sum(A[i])
        for j in range(len(A[i])):
            A[i][j] /= norm

    # Randomly initialize and normalize matrix O.
    O = [[rng.random() for i in range(D)] for j in range(L)]

    for i in range(len(O)):
        norm = sum(O[i])
        for j in range(len(O[i])):
            O[i][j] /= norm

    # Train an HMM with unlabeled data.
    HMM = HiddenMarkovModel(A, O)
    HMM.unsupervised_learning(X, N_iters)

    return HMM

## support functions

In [ ]:
########################################
# CS/CNS/EE 155 2018
# Problem Set 6
#
# Author:       Andrew Kang
# Description:  Set 6 HMM helper
########################################

import re
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from matplotlib import animation
from matplotlib.animation import FuncAnimation


####################
# WORDCLOUD FUNCTIONS
####################


def mask():
    # Parameters.
    r = 128
    d = 2 * r + 1

    # Get points in a circle.
    y, x = np.ogrid[-r : d - r, -r : d - r]
    circle = x ** 2 + y ** 2 <= r ** 2

    # Create mask.
    mask = 255 * np.ones((d, d), dtype=np.uint8)
    mask[circle] = 0

    return mask


def text_to_wordcloud(text, max_words=50, title="", show=True):
    plt.close("all")

    # Generate a wordcloud image.
    wordcloud = WordCloud(
        random_state=0, max_words=max_words, background_color="white", mask=mask()
    ).generate(text)

    # Show the image.
    if show:
        plt.figure(figsize=(10, 10))
        plt.imshow(wordcloud, interpolation="bilinear")
        plt.axis("off")
        plt.title(title, fontsize=24)
        plt.show()

    return wordcloud


def states_to_wordclouds(hmm, obs_map, max_words=100, show=True):
    # Initialize.
    M = 100000
    n_states = len(hmm.A)
    # print(n_states)
    obs_map_r = obs_map_reverser(obs_map)
    wordclouds = []

    # Generate a large emission.
    emission, states = hmm.generate_emission(M)

    # For each state, get a list of observations that have been emitted
    # from that state.
    obs_count = []
    for i in range(n_states):
        obs_lst = np.array(emission)[np.where(np.array(states) == i)[0]]
        obs_count.append(obs_lst)

    # For each state, convert it into a wordcloud.
    state_words = [None] * n_states
    state_counts = [None] * n_states

    for i in range(n_states):
        obs_lst = obs_count[i]
        sentence = [obs_map_r[j] for j in obs_lst]
        sentence_str = " ".join(sentence)

        # count number of words

        counted_words = collections.Counter(sentence)
        words = []
        counts = []
        for letter, count in counted_words.most_common(10):
            words.append(letter)
            counts.append(count)

        state_words[i] = words
        state_counts[i] = counts

        """plt.title('Top words in State %d vs their count' % i)
        plt.xlabel('Count')
        plt.ylabel('Words')
        plt.barh(words, counts)
        plt.gca().invert_yaxis()
        plt.show()"""

        try:
            wordclouds.append(
                text_to_wordcloud(
                    sentence_str, max_words=max_words, title="State %d" % i, show=show
                )
            )

        except:
            print(f"Skipping state %d" % i)

    return wordclouds, state_words, state_counts


####################
# HMM FUNCTIONS
####################


def parse_observations(text):
    # Convert text to dataset.
    lines = [line.split() for line in text.split("\n") if line.split()]

    obs_counter = 0
    obs = []
    obs_map = {}

    for line in lines:
        obs_elem = []

        for word in line:
            word = re.sub(r"[^\w]", "", word).lower()
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1

            # Add the encoded word.
            obs_elem.append(obs_map[word])

        # Add the encoded sequence.
        obs.append(obs_elem)

    return obs, obs_map


def obs_map_reverser(obs_map):
    obs_map_r = {}

    for key in obs_map:
        obs_map_r[obs_map[key]] = key

    return obs_map_r


def gen_word_list(hmm, obs_map, n_words=100):
    # Get reverse map.
    obs_map_r = obs_map_reverser(obs_map)

    # Sample and convert sentence.
    emission, states = hmm.generate_emission(n_words)

    return [obs_map_r[i] for i in emission]


####################
# HMM VISUALIZATION FUNCTIONS
####################


def visualize_sparsities(hmm, O_max_cols=50, O_vmax=0.1):
    plt.close("all")
    plt.set_cmap("viridis")

    # Visualize sparsity of A.
    plt.imshow(hmm.A, vmax=1.0)
    plt.colorbar()
    plt.title("Sparsity of A matrix")
    plt.show()

    # Visualize parsity of O.
    plt.imshow(np.array(hmm.O)[:, :O_max_cols], vmax=O_vmax, aspect="auto")
    plt.colorbar()
    plt.title("Sparsity of O matrix")
    plt.show()


####################
# HMM ANIMATION FUNCTIONS
####################


def animate_emission(hmm, obs_map, M=20, height=12, width=12, delay=1):
    # Parameters.
    lim = 1200
    text_x_offset = 40
    text_y_offset = 80
    x_offset = 580
    y_offset = 520
    R = 420
    r = 100
    arrow_size = 20
    arrow_p1 = 0.03
    arrow_p2 = 0.02
    arrow_p3 = 0.06

    # Initialize.
    n_states = len(hmm.A)
    obs_map_r = obs_map_reverser(obs_map)
    wordclouds, state_words, state_counts = states_to_wordclouds(
        hmm, obs_map, max_words=100, show=True
    )
    plot_top_words(state_words, state_counts)
    # Initialize plot.
    fig, ax = plt.subplots()
    fig.set_figheight(height)
    fig.set_figwidth(width)
    ax.grid("off")
    plt.axis("off")
    ax.set_xlim([0, lim])
    ax.set_ylim([0, lim])

    # Plot each wordcloud.
    for i, wordcloud in enumerate(wordclouds):
        x = x_offset + int(R * np.cos(np.pi * 2 * i / n_states))
        y = y_offset + int(R * np.sin(np.pi * 2 * i / n_states))
        ax.imshow(
            wordcloud.to_array(),
            extent=(x - r, x + r, y - r, y + r),
            aspect="auto",
            zorder=-1,
        )

    # Initialize text.
    text = ax.text(text_x_offset, lim - text_y_offset, "", fontsize=24)

    # Make the arrows.
    zorder_mult = n_states ** 2 * 100
    arrows = []
    for i in range(n_states):
        row = []
        for j in range(n_states):
            # Arrow coordinates.
            x_i = x_offset + R * np.cos(np.pi * 2 * i / n_states)
            y_i = y_offset + R * np.sin(np.pi * 2 * i / n_states)
            x_j = x_offset + R * np.cos(np.pi * 2 * j / n_states)
            y_j = y_offset + R * np.sin(np.pi * 2 * j / n_states)

            dx = x_j - x_i
            dy = y_j - y_i
            d = np.sqrt(dx ** 2 + dy ** 2)

            if i != j:
                arrow = ax.arrow(
                    x_i + (r / d + arrow_p1) * dx + arrow_p2 * dy,
                    y_i + (r / d + arrow_p1) * dy + arrow_p2 * dx,
                    (1 - 2 * r / d - arrow_p3) * dx,
                    (1 - 2 * r / d - arrow_p3) * dy,
                    color=(1 - hmm.A[i][j],) * 3,
                    head_width=arrow_size,
                    head_length=arrow_size,
                    zorder=int(hmm.A[i][j] * zorder_mult),
                )
            else:
                arrow = ax.arrow(
                    x_i,
                    y_i,
                    0,
                    0,
                    color=(1 - hmm.A[i][j],) * 3,
                    head_width=arrow_size,
                    head_length=arrow_size,
                    zorder=int(hmm.A[i][j] * zorder_mult),
                )

            row.append(arrow)
        arrows.append(row)

    emission, states = hmm.generate_emission(M)

    def animate(i):
        if i >= delay:
            i -= delay

            if i == 0:
                arrows[states[0]][states[0]].set_color("red")
            elif i == 1:
                arrows[states[0]][states[0]].set_color(
                    (1 - hmm.A[states[0]][states[0]],) * 3
                )
                arrows[states[i - 1]][states[i]].set_color("red")
            else:
                arrows[states[i - 2]][states[i - 1]].set_color(
                    (1 - hmm.A[states[i - 2]][states[i - 1]],) * 3
                )
                arrows[states[i - 1]][states[i]].set_color("red")

            # Set text.
            text.set_text(
                " ".join([obs_map_r[e] for e in emission][: i + 1]).capitalize()
            )

            return arrows + [text]

    # Animate!
    print("\nAnimating...")
    anim = FuncAnimation(fig, animate, frames=M + delay, interval=1000)

    return anim

    # honestly this function is so jank but who even fuckin cares
    # i don't even remember how or why i wrote this mess
    # no one's gonna read this
    # hey if you see this tho hmu on fb let's be friends

In [ ]:
class Utility:
    """
    Utility for the problem files.
    """

    def __init__():
        pass

    @staticmethod
    def load_sequence(n):
        """
        Load the file 'sequence_data<n>.txt' for a given n.
        Arguments:
            n:          Sequence index.
        Returns:
            A:          The transition matrix.
            O:          The observation matrix.
            seqs:       Input sequences.
        """
        A = []
        O = []
        seqs = []

        # For each file:
        with urllib.request.urlopen(
            f"https://raw.githubusercontent.com/lakigigar/Caltech-CS155-2021/main/psets/set6/data/sequence_data{n}.txt"
        ) as f:
            # Read the parameters.
            L, D = [int(x) for x in f.readline().decode("utf-8").strip().split("\t")]

            # Read the transition matrix.
            for i in range(L):
                A.append(
                    [float(x) for x in f.readline().decode("utf-8").strip().split("\t")]
                )

            # Read the observation matrix.
            for i in range(L):
                O.append(
                    [float(x) for x in f.readline().decode("utf-8").strip().split("\t")]
                )

            # The rest of the file consists of sequences.
            while True:
                seq = f.readline().decode("utf-8").strip()
                if seq == "":
                    break
                seqs.append([int(x) for x in seq])

        return A, O, seqs

    @staticmethod
    def load_ron():
        """
        Loads the file 'ron.txt'.
        Returns:
            moods:      Sequnces of states, i.e. a list of lists.
                        Each sequence represents half a year of data.
            mood_map:   A hash map that maps each state to an integer.
            genres:     Sequences of observations, i.e. a list of lists.
                        Each sequence represents half a year of data.
            genre_map:  A hash map that maps each observation to an integer.
        """
        moods = []
        mood_map = {}
        genres = []
        genre_map = {}
        mood_counter = 0
        genre_counter = 0

        with urllib.request.urlopen(
            "https://raw.githubusercontent.com/lakigigar/Caltech-CS155-2021/main/psets/set6/data/ron.txt"
        ) as f:
            mood_seq = []
            genre_seq = []

            while True:
                line = f.readline().decode("utf-8").strip()

                if line == "" or line == "-":
                    # A half year has passed. Add the current sequence to
                    # the list of sequences.
                    moods.append(mood_seq)
                    genres.append(genre_seq)
                    # Start new sequences.
                    mood_seq = []
                    genre_seq = []

                if line == "":
                    break
                elif line == "-":
                    continue

                mood, genre = line.split()

                # Add new moods to the mood state hash map.
                if mood not in mood_map:
                    mood_map[mood] = mood_counter
                    mood_counter += 1

                mood_seq.append(mood_map[mood])

                # Add new genres to the genre observation hash map.
                if genre not in genre_map:
                    genre_map[genre] = genre_counter
                    genre_counter += 1

                # Convert the genre into an integer.
                genre_seq.append(genre_map[genre])

        return moods, mood_map, genres, genre_map

    @staticmethod
    def load_ron_hidden():
        """
        Loads the file 'ron.txt' and hides the states.
        Returns:
            genres:     The observations.
            genre_map:  A hash map that maps each observation to an integer.
        """
        moods, mood_map, genres, genre_map = Utility.load_ron()

        return genres, genre_map

## load text and use pre-process keys

In [ ]:
import pickle

In [ ]:
text = (
    urllib.request.urlopen(
        "https://raw.githubusercontent.com/lakigigar/Caltech-CS155-2021/main/projects/project3/data/shakespeare.txt"
    )
    .read()
    .decode("utf-8")
)
wordcloud = text_to_wordcloud(text, title="shakespeare")

## train unsupervised HMM for quatrain, volta, couplet

In [ ]:
quatrain,volta,couplet = load_data()
key_list = load_key()

seq_lists,key_lists = generate_sequences_reduced()

quatrain,volta,couplet = load_data()
key_list = load_key(includePunctuation=False)

rhymes = get_rhymes(generate_sequences_reduced(includePunctuation=False)[0])
rhyme_words = generate_rhyme_lists()

In [ ]:
def key_list2obs_map(key_lists):
    """Convert key_lists to the same format as obs_map"""
    obs_map_dict = {}
    hmm_names = ["quatrain", "volta", "couplet"]

    for name, key_list in zip(hmm_names, key_lists):
        obs_map = {}
        for i, k in enumerate(key_list):
            obs_map[k[0]] = i
        obs_map_dict[name] = obs_map
    return obs_map_dict


def del_empty_seq_list(seq_list):
    """Get rid of empty lists"""
    return [s for s in seq_list if len(s) != 0]


def train_3hmm(seq_lists, n_states=10, N_iters=100):
    """Train separate hmm for quatrain, volta, and couplet"""
    hmm_dict = {}
    hmm_names = ["quatrain", "volta", "couplet"]
    for name, seq_list in zip(hmm_names, seq_lists):
        seq_list = del_empty_seq_list(seq_list)
        hmm_dict[name] = unsupervised_HMM(seq_list, n_states, N_iters)
    return hmm_dict

### different number of hidden states

In [ ]:
hmm2_100_dict = train_3hmm(seq_lists, n_states=2, N_iters=100)

In [ ]:
hmm5_100_dict = train_3hmm(seq_lists, n_states=5, N_iters=100)

In [ ]:
hmm10_100_dict = train_3hmm(seq_lists, n_states=10, N_iters=100)

In [ ]:
hmm20_100_dict = train_3hmm(seq_lists, n_states=20, N_iters=100)

### different number of iterations

In [ ]:
hmm10_500_dict = train_3hmm(seq_lists, n_states=10, N_iters=500)

In [ ]:
hmm10_1000_dict = train_3hmm(seq_lists, n_states=10, N_iters=1000)

In [ ]:
hmm20_500_dict = train_3hmm(seq_lists, n_states=20, N_iters=500)

In [ ]:
hmm20_1000_dict = train_3hmm(seq_lists, n_states=20, N_iters=1000)

## train unsupervised HMM as a whole (no punc)

In [ ]:
# obs, obs_map = parse_observations(text)

In [ ]:
# HMM10 = unsupervised_HMM(obs, 10, 100)

## save and load trained HMM

### save

In [ ]:
"""with open("unsupervised_qvcwpunc_HMM_2_hidenstate_100_iter.pkl", "wb") as f:
    pickle.dump(hmm2_100_dict, f)
with open("unsupervised_qvcwpunc_HMM_5_hidenstate_100_iter.pkl", "wb") as f:
    pickle.dump(hmm5_100_dict, f)
with open("unsupervised_qvcwpunc_HMM_20_hidenstate_100_iter.pkl", "wb") as f:
    pickle.dump(hmm20_100_dict, f)"""

In [ ]:
"""with open("unsupervised_qvcwpunc_HMM_20_hidenstate_500_iter.pkl", "wb") as f:
    pickle.dump(hmm20_500_dict, f)
with open("unsupervised_qvcwpunc_HMM_20_hidenstate_1000_iter.pkl", "wb") as f:
    pickle.dump(hmm20_1000_dict, f)"""

In [ ]:
"""with open("unsupervised_qvcwpunc_HMM_10_hidenstate_100_iter.pkl", "wb") as f:
    pickle.dump(hmm10_100_dict, f)
with open("unsupervised_qvcwpunc_HMM_10_hidenstate_500_iter.pkl", "wb") as f:
    pickle.dump(hmm10_500_dict, f)
with open("unsupervised_qvcwpunc_HMM_10_hidenstate_1000_iter.pkl", "wb") as f:
    pickle.dump(hmm10_1000_dict, f)"""

In [ ]:
"""with open("unsupervised_HMM_10_hidenstate.pkl", "wb") as f:
    pickle.dump(HMM10, f)"""

### load

In [ ]:
with open("unsupervised_qvcwpunc_HMM_2_hidenstate_100_iter.pkl", "rb") as f:
    hmm2_100_dict = pickle.load(f)
with open("unsupervised_qvcwpunc_HMM_5_hidenstate_100_iter.pkl", "rb") as f:
    hmm5_100_dict = pickle.load(f)
with open("unsupervised_qvcwpunc_HMM_20_hidenstate_100_iter.pkl", "rb") as f:
    hmm20_100_dict = pickle.load(f)

In [ ]:
with open("unsupervised_qvcwpunc_HMM_20_hidenstate_500_iter.pkl", "rb") as f:
    hmm20_500_dict = pickle.load(f)
with open("unsupervised_qvcwpunc_HMM_20_hidenstate_1000_iter.pkl", "rb") as f:
    hmm20_1000_dict = pickle.load(f)

In [ ]:
with open("unsupervised_qvcwpunc_HMM_10_hidenstate_100_iter.pkl", "rb") as f:
    hmm10_100_dict = pickle.load(f)
with open("unsupervised_qvcwpunc_HMM_10_hidenstate_500_iter.pkl", "rb") as f:
    hmm10_500_dict = pickle.load(f)
with open("unsupervised_qvcwpunc_HMM_10_hidenstate_1000_iter.pkl", "rb") as f:
    hmm10_1000_dict = pickle.load(f)

In [ ]:
with open("unsupervised_HMM_10_hidenstate.pkl", "rb") as f:
    HMM10 = pickle.load(f)

## visualize A and O

In [ ]:
def vis_hmm_dict(hmm_dict):
    """Visulize the A and O matrix for quatrain, volta, and couplet"""
    for k, v in hmm_dict.items():
        print(f"Visulize A and O for {k}")
        visualize_sparsities(v, O_max_cols=50)

In [ ]:
visualize_sparsities(HMM10, O_max_cols=50)

In [ ]:
print("Vis 2 hidden states, 100 iterations\n")
vis_hmm_dict(hmm2_100_dict)
print("Vis 5 hidden states, 100 iterations\n")
vis_hmm_dict(hmm5_100_dict)
print("Vis 10 hidden states, 100 iterations\n")
vis_hmm_dict(hmm10_100_dict)
print("Vis 20 hidden states, 100 iterations\n")
vis_hmm_dict(hmm20_100_dict)

In [ ]:
print("Vis 10 hidden states, 500 iterations\n")
vis_hmm_dict(hmm10_500_dict)
print("Vis 10 hidden states, 1000 iterations\n")
vis_hmm_dict(hmm10_1000_dict)

In [ ]:
print("Vis 20 hidden states, 500 iterations\n")
vis_hmm_dict(hmm20_500_dict)
print("Vis 20 hidden states, 1000 iterations\n")
vis_hmm_dict(hmm20_1000_dict)

## generation

In [ ]:
import itertools

In [ ]:
def gen_aline(
    hmm, obs_map, key_list, punc_list=[",", ".", "?", ":", ";", "!"], n_words=15
):
    """"""
    aline = gen_word_list(hmm, obs_map, n_words=n_words)
    syll_numb = 0
    output_list = []
    output_sentence = ""
    for i, word in enumerate(aline):
        # don't want punc other than ( at the beginning of a sentence
        if len(output_list) == 0 and word in punc_list:
            continue
        # don't want multiple punc together
        elif len(output_list) > 0 and output_list[-1] in punc_list and word in punc_list:
            continue
        # don't want same words together
        elif len(output_list) > 0 and output_list[-1] == word:
            continue

        else:
            find_syll = [key for key in key_list if word in key]
            if len(find_syll) != 0:

                # fix I in the middle of the sentence
                if word == "i":
                    word = "I"

                for possible_syll in find_syll[0][1]:
                    if (
                        syll_numb + possible_syll < 10
                        and syll_numb + find_syll[0][2] < 10
                    ):
                        syll_numb += possible_syll
                        output_list.append(word)
                        break

                    elif syll_numb + possible_syll == 10 and find_syll[0][2] == 0:
                        syll_numb += possible_syll
                        output_list.append(word)
                        if i < len(aline) - 1:
                            next_letter = aline[i + 1]
                            if word not in punc_list and next_letter in punc_list:
                                output_list.append(next_letter)
                        break

                    elif syll_numb + find_syll[0][2] == 10 and find_syll[0][2] != 0:
                        syll_numb += find_syll[0][2]
                        output_list.append(word)
                        if i < len(aline) - 1:
                            next_letter = aline[i + 1]
                            if next_letter in punc_list:
                                output_list.append(next_letter)
                        break

    return syll_numb, output_list

In [ ]:
def gen_poem(hmm, obs_map, key_list, n_words=15):
    """Generate the 14 lines"""
    poem = [None] * 14
    line_count = 0
    while line_count < 14:
        syll_numb, output_list = gen_aline(hmm, obs_map, key_list, n_words=n_words)
        if syll_numb == 10:
            poem[line_count] = " ".join(output_list).capitalize()
            line_count = line_count + 1

    return poem

In [ ]:
def print_poem(poem):
    """Print the poem with given format with non-optimized punctuations"""
    for i, p in enumerate(poem[:-2]):
        if (i + 1) % 4 == 0:
            print(p + ".")
        else:
            print(p + ",")

    print("  " + poem[-2] + ",")
    print("  " + poem[-1] + ".")

In [ ]:
def gen_qvc(
    hmm_dict,
    obs_map_dict,
    key_lists,
    qvc,
    punc_list=[",", ".", "?", ":", ";", "!"],
    end_punc_list=[".", "?"],
    n_words=15,
):
    """Generate 4 lines of quatrain or volta or 2 lines of couplet"""
    if qvc == "couplet":
        line_needed = 2
    else:
        line_needed = 4

    poem_qcv = [None] * line_needed
    line_count = 0
    while line_count < line_needed:
        syll_numb, output_list = gen_aline(
            hmm_dict[qvc], obs_map_dict[qvc], key_lists[0], n_words=12
        )
        if syll_numb == 10:
            output_sentence = ""
            for i, word in enumerate(output_list):
                if i == 0:
                    output_sentence += word.capitalize()
                elif word in punc_list:
                    output_sentence += word
                elif i > 0 and output_list[i - 1] in end_punc_list:
                    output_sentence = output_sentence + " " + word.capitalize()
                else:
                    output_sentence = output_sentence + " " + word
            poem_qcv[line_count] = output_sentence
            line_count = line_count + 1

    return poem_qcv

In [ ]:
def comb_qvc(hmm_dict, obs_map_dict, key_lists, n_words=15):
    """Combine generated qvc"""
    poem = [None] * 4
    poem[0] = gen_qvc(hmm_dict, obs_map_dict, key_lists, "quatrain", n_words=n_words)
    poem[1] = gen_qvc(hmm_dict, obs_map_dict, key_lists, "quatrain", n_words=n_words)
    poem[2] = gen_qvc(hmm_dict, obs_map_dict, key_lists, "volta", n_words=n_words)
    poem[3] = gen_qvc(hmm_dict, obs_map_dict, key_lists, "couplet", n_words=n_words)
    return list(itertools.chain.from_iterable(poem))

In [ ]:
def print_qvc(poem):
    """Print the poem in qvc order with given format"""
    for i, p in enumerate(poem[:-2]):
        print(p)

    print("  " + poem[-2])
    print("  " + poem[-1])
    print("\n")

In [ ]:
obs_map_dict = key_list2obs_map(key_lists)

In [ ]:
for i in range(10):
    print_qvc(comb_qvc(hmm2_100_dict, obs_map_dict, key_lists))

In [ ]:
for i in range(10):
    print_qvc(comb_qvc(hmm5_100_dict, obs_map_dict, key_lists))

In [ ]:
for i in range(10):
    print_qvc(comb_qvc(hmm20_100_dict, obs_map_dict, key_lists, n_words=20))

In [ ]:
print_qvc(comb_qvc(hmm10_500_dict, obs_map_dict, key_lists))

In [ ]:
for i in range(10):
    print_qvc(comb_qvc(hmm10_500_dict, obs_map_dict, key_lists))

In [ ]:
for i in range(10):
    print_qvc(comb_qvc(hmm10_1000_dict, obs_map_dict, key_lists))

In [ ]:
for i in range(10):
    print_qvc(comb_qvc(hmm10_1000_dict, obs_map_dict, key_lists))

In [ ]:
for i in range(10):
    print_qvc(comb_qvc(hmm10_1000_dict, obs_map_dict, key_lists))

In [ ]:
for i in range(10):
    print_qvc(comb_qvc(hmm10_1000_dict, obs_map_dict, key_lists))

In [ ]:
for i in range(10):
    print_qvc(comb_qvc(hmm20_500_dict, obs_map_dict, key_lists, n_words=20))

In [ ]:
for i in range(10):
    print_qvc(comb_qvc(hmm20_1000_dict, obs_map_dict, key_lists, n_words=20))

In [ ]:
for i in range(10):
    print_poem(gen_poem(HMM10, obs_map, key_list, n_words=12))
    print("\n")

In [ ]:
from IPython.display import HTML

In [ ]:
def wc_am_vis(hmms, obs_map_dict, ifanim=True):
    """Word cloud and animiation"""
    for qvc, hmm in hmms.items():
        print(f"Vis {qvc}")
        """wordclouds, state_words, state_counts = states_to_wordclouds(
            hmm, obs_map_dict[qvc]
        )
        plot_top_words(state_words, state_counts)
        if ifanim:"""
        anim = animate_emission(hmm, obs_map_dict[qvc], M=20)
        plt.show()
        HTML(anim.to_html5_video())

In [ ]:
def wc_vis(hmms, obs_map_dict):
    """Word cloud and animiation"""
    for qvc, hmm in hmms.items():
        print(f"Vis {qvc}")
        wordclouds, state_words, state_counts = states_to_wordclouds(
            hmm, obs_map_dict[qvc]
        )
        plot_top_words(state_words, state_counts)

In [ ]:
def am_vis(hmms, obs_map_dict):
    """Word cloud and animiation"""
    for qvc, hmm in hmms.items():
        print(f"Anim {qvc}")
        anim = animate_emission(hmm, obs_map_dict[qvc], M=20)
        plt.show()
        HTML(anim.to_html5_video())
        

In [ ]:
def plot_top_words(state_words, state_counts):
    """Plot the bar graphs"""
    for i, w in enumerate(state_words):
        plt.title('Top words in State %d vs their count' % i)
        plt.xlabel('Count')
        plt.ylabel('Words')
        plt.barh(w, state_counts[i])
        plt.gca().invert_yaxis()
        plt.show()

In [ ]:
wc_am_vis(hmm2_100_dict, obs_map_dict)

In [ ]:
wc_am_vis(hmm5_100_dict, obs_map_dict)

In [ ]:
wc_am_vis(hmm10_100_dict, obs_map_dict)
wc_am_vis(hmm10_500_dict, obs_map_dict)
wc_am_vis(hmm10_1000_dict, obs_map_dict)

In [ ]:
wc_am_vis(hmm20_100_dict, obs_map_dict)
wc_am_vis(hmm20_500_dict, obs_map_dict)
wc_am_vis(hmm20_1000_dict, obs_map_dict)

In [ ]:
"""am_vis(hmm2_100_dict, obs_map_dict)
am_vis(hmm5_100_dict, obs_map_dict)

am_vis(hmm10_100_dict, obs_map_dict)
am_vis(hmm10_500_dict, obs_map_dict)
am_vis(hmm10_1000_dict, obs_map_dict)

am_vis(hmm20_100_dict, obs_map_dict)
am_vis(hmm20_500_dict, obs_map_dict)
am_vis(hmm20_1000_dict, obs_map_dict)"""